In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the dataset
# Preparing the training set and the test set
training_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AutoEncoders/ml-100k/ml-100k/u1.base', delimiter = '\t')
test_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AutoEncoders/ml-100k/ml-100k/u1.test', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [3]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [4]:
# Training the SAE
nb_epoch = 200
for epoch in range (1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0 :
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print("epoch: "+ str(epoch) + " loss: "+str(train_loss/s))


epoch: 1 loss: 1.7715553558755277
epoch: 2 loss: 1.0966505167726368
epoch: 3 loss: 1.0533941376079667
epoch: 4 loss: 1.038590427021636
epoch: 5 loss: 1.0310734254618317
epoch: 6 loss: 1.026732199711667
epoch: 7 loss: 1.0239840901229535
epoch: 8 loss: 1.0218837915540713
epoch: 9 loss: 1.0204988627812355
epoch: 10 loss: 1.0198719865394514
epoch: 11 loss: 1.0186794137134871
epoch: 12 loss: 1.0185331186594
epoch: 13 loss: 1.0178460384867232
epoch: 14 loss: 1.017630415125318
epoch: 15 loss: 1.0172332084209508
epoch: 16 loss: 1.0167501593492698
epoch: 17 loss: 1.0165770549333693
epoch: 18 loss: 1.0163514725319465
epoch: 19 loss: 1.0164293907494835
epoch: 20 loss: 1.0162901845862067
epoch: 21 loss: 1.0158124642142587
epoch: 22 loss: 1.0159370902922158
epoch: 23 loss: 1.0159075361047232
epoch: 24 loss: 1.0159775529963702
epoch: 25 loss: 1.0155825166944559
epoch: 26 loss: 1.0156439168450617
epoch: 27 loss: 1.015278358749521
epoch: 28 loss: 1.0152073700426174
epoch: 29 loss: 1.0138848021544493
e

In [5]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0 :
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data.item()*mean_corrector)
        s += 1.
        optimizer.step()
print("test_loss: "+str(test_loss/s))

test_loss: 0.9535596982476557
